In [ ]:
!pip install datasets==3.6.0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer

from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
reviews = pd.read_csv("imdb.csv").sample(500)
reviews.head()

In [ ]:
reviews = reviews.rename(columns={"review": "text", "sentiment": "label"})
reviews['label'] = LabelEncoder().fit_transform(reviews['label'])
reviews.head()

To help with this task, we're going to use the [Datasets library](https://huggingface.co/docs/datasets/en/index), which allows for working with various types of datasets, from text, to audio, to images.

First, convert the reviews DataFrame into a dataset object by using the [from_pandas function](https://huggingface.co/docs/datasets/en/index).

In [ ]:
# Your Code Here

Once converted, we can perform a train/test split using a method of the Dataset object. 

Peform an 80/20 train/test split using the [train_test_split method](https://huggingface.co/docs/datasets/en/index). Save the result back to the same Dataset object.

In [ ]:
# Your Code Here

Now, extract the training and test portion into separate Datasets. Name these new datasets train_dataset and test_dataset, respectively.

In [ ]:
# Your Code Here

We're going to be working with a DistilBERT model, which means that we'll need to tokenize our input in the way that DistilBERT expects. For this, we can use the [DistilBertTokenizerFast](https://huggingface.co/docs/transformers/en/model_doc/distilbert?usage=AutoModel#transformers.DistilBertTokenizerFast) tokenizer.

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

Then we'll apply our tokenizer to the training and test datsets using the map method.

In [ ]:
train_dataset = train_dataset.map(lambda df: tokenizer(df['text'], padding="max_length", truncation=True), batched=True)
test_dataset = test_dataset.map(lambda df: tokenizer(df['text'], padding="max_length", truncation=True), batched=True)

Finally, we'll set the format to torch so that we're working with [PyTorch](https://pytorch.org/) tensors and only extract the columns that we need.

In [ ]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Now, let's load in the pretrained DistilBERT model.

### Part 1: Fine-tuning All Parameters

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Now, we need to set up a [Trainer](https://huggingface.co/docs/transformers/en/main_classes/trainer) for our model.

First, create a [TraininingArguments](https://huggingface.co/docs/transformers/v4.52.3/en/main_classes/trainer#transformers.TrainingArguments) object. Set num_train_epochs to 5, weight_decay to 0.01, and report_to = 'none'

In [ ]:
# Your Code Here

Finally, create a Trainer object using the model, the Training Arguments that you created, and with the train_dataset equal to train_dataset.

In [ ]:
# Your Code Here

Now, use the [train method](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Trainer.train) on your Trainer object.

In [ ]:
#Your Code Here

Once the model has been fit, use the [predict method](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Trainer.predict) of the Trainer object to generate a set of predictions on the test_dataset.

In [ ]:
#Your Code Here

Extract the actual test labels and the predicted labels from the predictions. Note that both the true labels and predicted probabilites are contained as an attribute of the predictions.

In [ ]:
#Your Code Here

Finally, look the confusion matrix and classification report for these predictions.

In [ ]:
#Your Code Here

### Part 2: Training Only a Subset of the Parameters

Let's first reload the pretrained distilbert model.

Then, we'll make none of the parameters trainable by setting the `requires_grad` attribute to False.

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

for param in model.distilbert.parameters():
    param.requires_grad = False

Then, we'll go back and make the last 2 layers trainable.

In [ ]:
for i in [4, 5]:
    for param in model.distilbert.transformer.layer[i].parameters():
        param.requires_grad = True

We'll now set up TrainingArguments and a Trainer as before to train the model.

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    weight_decay=0.01,
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

How do the results of fine-tuning only the last two layers compare to fine-tuning all layers? How does the training time compare?

In [ ]:
#Your Code Here

### Part 3: Parameter-Efficient Fine-Tuning

Now, let's see how we can use the [peft library](https://huggingface.co/docs/peft/en/index) to more efficiently fine-tune our model.

First, we'll re-initalize the model.

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Create a [LoraConfig object](https://huggingface.co/docs/peft/en/index) were you set the Lora attention dimension to 8, the lora_alpha to 16, the lora_dropout to 0.1, the target_modules to "q_lin" and "v_lin" (these are the "query" and "value" projections), and set the task_type to TaskType.SEQ_CLS.

In [ ]:
#Your Code Here

Now, use the [get_peft_model function](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.get_peft_model) to create the Lora model pass in the distilbert model and the LoraConfig object that you created.

In [ ]:
#Your Code Here

How many trainable parameters does the resulting model have? Hint: you can use the [print_trainable_parameters function](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.PeftModel.print_trainable_parameters).

In [ ]:
#Your Code Here

We'll again set up the same TrainingArguments and create our Trainer object to grain the model.

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    weight_decay=0.01,
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

How does the performance of the lora model compare to the previous two? How does the training time compare?

In [ ]:
#Your Code Here